# Notebook: Clustering com K-Means, DBSCAN, Hierárquico e Detecção de Anomalias

## Altere o código abaixo para: realizar um loop para diferentes valores de K para a instância do algoritmo KMeans

In [ ]:
### Seção 1: Explorando o número de clusters no K-Means

from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np

# Gerar dados com 4 clusters
X, y = make_blobs(n_samples=300, centers=4, cluster_std=0.6, random_state=0)

### 🔍 Atividade Investigativa:
# Execute o K-Means com múltiplos valores de k = 3, 4, 5 e 6.
# Para cada valor de k:
# - Visualize os resultados
# - Marque visualmente os centróides
# - Observe se há pontos mal agrupados ou clusters desbalanceados
# - Anote suas observações e hipóteses

k=1

kmeans = KMeans(n_clusters=k, random_state=0).fit(X)
plt.scatter(X[:, 0], X[:, 1], c=kmeans.labels_, cmap='viridis')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s=200, c='red', marker='X')
plt.title(f'K-Means com {k} clusters')
plt.show()

### Exercício:
# Responda em grupo ou individualmente:
# - Qual valor de k gerou agrupamentos mais coerentes? Por quê?
# - Em qual caso você viu mais pontos mal alocados?
# - Como você escolheria um valor de k se não soubesse os rótulos verdadeiros?


### 🌀 Inércia (KMeans)
Definição:
A inércia mede o quão perto os pontos estão dos seus respectivos centroides no KMeans. É a soma das distâncias quadradas entre os pontos e o centro do seu cluster.

Interpretação:

Quanto menor a inércia, mais compactos são os clusters.

Não é uma métrica absoluta de "qualidade", mas ajuda a comparar modelos com diferentes números de clusters (k).

Usamos ela no método do cotovelo para escolher o melhor valor de k.



### 🧩 Índice de Silhueta (Silhouette Score)
Definição:
O índice de silhueta mede o quão bem cada ponto está agrupado em comparação com outros clusters.
 
Valores:

Varia de -1 a 1:

Próximo de 1 → ponto bem agrupado.

Próximo de 0 → ponto na fronteira entre clusters.

Próximo de -1 → ponto provavelmente no cluster errado.

Uso:

Avaliar a qualidade geral dos agrupamentos.

Comparar diferentes algoritmos ou valores de k mesmo com formas de cluster irregulares.

## Novamente aqui, selecione múltiplos valores de K (2 a 10)

In [ ]:
### Seção 2: Escolha do melhor K com Inércia e Silhouette

from sklearn.metrics import silhouette_score

inertias = []
silhouettes = []

k=2

# Avaliar vários valores de K
model = KMeans(n_clusters=k, random_state=0).fit(X)
inertias.append(model.inertia_)
silhouettes.append(silhouette_score(X, model.labels_))

# Visualizar curvas
plt.plot(ks, inertias, marker='o')
plt.title('Inércia vs. K')
plt.xlabel('K')
plt.ylabel('Inércia')
plt.show()

plt.plot(ks, silhouettes, marker='o')
plt.title('Silhouette Score vs. K')
plt.xlabel('K')
plt.ylabel('Silhouette')
plt.show()

### Exercício:
# - Identifique o melhor valor de K com base nos gráficos.
# - Justifique sua escolha com base na variação da inércia e do Silhouette Score.

### Tempo estimado: 10 minutos




### 🖼️ Compressão de Imagem com K-Means
🤔 O que é?
A compressão de imagem com K-Means é uma técnica que reduz o número de cores distintas em uma imagem, mantendo a aparência visual parecida. Isso é feito agrupando cores semelhantes e substituindo todas as cores de um grupo pela cor média (centróide).

⚙️ Como funciona?
Cada pixel da imagem é representado como um vetor RGB (ex: [255, 0, 0] para vermelho).

Aplicamos o K-Means nesses vetores de cores para encontrar k clusters de cores.

Cada pixel é então substituído pela cor do centróide do cluster ao qual ele pertence.

Resultado: uma imagem visualmente semelhante, mas com menos cores → menor tamanho em disco.

### 📉 Por que isso é útil?
Compressão sem perder muito a qualidade visual: útil em web, dispositivos móveis e jogos.

Redução de uso de memória e largura de banda.

Ajuda em tarefas de pré-processamento de imagem, removendo ruído de cor.

Aplicável a estilização de imagens e efeitos artísticos (tipo pôster com 5 cores).



In [ ]:
### Seção 3: Compressão de imagem com K-Means

from skimage import io

# Carregar imagem de exemplo
img = io.imread('pesQJS8.jpeg') / 255
img_data = img.reshape(-1, 3)

# Aplicar K-Means para quantização de cores
kmeans_img = KMeans(n_clusters=3, random_state=0).fit(img_data)
new_colors = kmeans_img.cluster_centers_[kmeans_img.labels_]
compressed_img = new_colors.reshape(img.shape)

plt.imshow(compressed_img)
plt.title('Imagem com cores reduzidas (K=8)')
plt.axis('off')
plt.show()

### Exercício:
# - Teste com 4, 8 e 16 cores.
# - Compare visualmente as compressões.
# - O que se perde? O que se mantém?



## DBSCAN

### 1. eps (epsilon)
O que é: o raio de vizinhança em torno de cada ponto.

Como funciona: para cada ponto P, o algoritmo conta quantos outros pontos estão a uma distância ≤ eps de P.

Impacto prático:

eps muito pequeno → poucos (ou nenhum) vizinhos dentro do raio → muitos pontos serão marcados como ruído (outliers).

eps muito grande → as regiões densas se fundem e pode acabar com apenas um cluster gigante.

Como escolher:

Calcule para cada ponto a distância até seu k-ésimo vizinho mais próximo (k = min_samples).

Ordene essas distâncias em ordem decrescente e trace o k-distance plot.

Procure o “joelho” (inflexão) do gráfico — aquele valor de distância costuma ser um bom eps.

### 2. min_samples
O que é: o número mínimo de pontos dentro do raio eps para que um ponto seja considerado um ponto de núcleo (core point).

Como funciona:

Se um ponto P tiver pelo menos min_samples (incluindo ele mesmo) dentro de sua vizinhança eps, P é núcleo.

Pontos vizinhos de um núcleo podem ser “alcançados” e fazem parte do mesmo cluster.

Pontos sem vizinhos suficientes e que não fazem parte da vizinhança de nenhum núcleo tornam-se ruído.

Impacto prático:

min_samples baixo (ex: 2 ou 3) → clusters sensíveis a pequenas aglomerações; pode criar muitos clusters pequenos.

min_samples alto → requer regiões bem densas para formar cluster; aumenta a chance de classificar pontos como ruído.

Como escolher:

Regra prática: min_samples ≈ D + 1, onde D é o número de dimensões dos seus dados.

Em geral, valores entre 4 e 10 costumam funcionar bem para dados em 2–3 dimensões.

Sempre valide visualmente (ou com métricas) e teste alguns valores.

In [ ]:
### Seção 4: Clustering com DBSCAN (agrupamento por densidade)

from sklearn.datasets import make_moons
from sklearn.cluster import DBSCAN

# Gerar dados em forma de lua
X_moons, _ = make_moons(n_samples=300, noise=0.05, random_state=0)

# Aplicar DBSCAN
# eps: raio de vizinhança, min_samples: mínimo de pontos por grupo denso
db = DBSCAN(eps=0.2, min_samples=5).fit(X_moons)

# Visualizar clusters encontrados
plt.scatter(X_moons[:, 0], X_moons[:, 1], c=db.labels_, cmap='rainbow')
plt.title('Clustering com DBSCAN')
plt.show()

### Exercício:
# - Altere os parâmetros eps e min_samples.
# - Observe como os clusters e outliers mudam.
# - Quais valores parecem razoáveis? Como saber?



In [ ]:
### Seção 5: Clustering Hierárquico

from sklearn.datasets import make_blobs
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering

# Gerar dados
X_hier, _ = make_blobs(n_samples=200, centers=4, cluster_std=0.7, random_state=42)

# Dendrograma
linkage_matrix = linkage(X_hier, method='ward')
plt.figure(figsize=(10, 5))
dendrogram(linkage_matrix)
plt.title('Dendrograma')
plt.xlabel('Amostras')
plt.ylabel('Distância')
plt.show()

# Clustering aglomerativo
agg = AgglomerativeClustering(n_clusters=4).fit(X_hier)
plt.scatter(X_hier[:, 0], X_hier[:, 1], c=agg.labels_, cmap='tab10')
plt.title('Agrupamento Hierárquico')
plt.show()

### Exercício:
# - O que o dendrograma mostra?
# - Como você decidiria o número de clusters com base no dendrograma?
# - Compare o resultado com o K-Means


## 📌 Detecção de Anomalias
### 💡 O que são anomalias?
Anomalias (ou outliers) são pontos que se comportam de forma significativamente diferente do padrão geral dos dados. Detectá-los é essencial em aplicações como:

Fraudes financeiras

Falhas em sensores ou máquinas

Comportamentos incomuns de usuários (cybersegurança, churn etc.)

Dados corrompidos

### 🧠 Como detectar anomalias com K-Means?
O K-Means assume que cada cluster tem uma forma esférica em torno de seu centróide. Uma ideia simples de anomalia:

Calcule a distância de cada ponto ao centro do cluster a que pertence.

Defina um limite (threshold) com base em percentis, como os 5% mais distantes.

Os pontos além desse limite são considerados anomalias.

➡️ Intuição: se um ponto está muito longe do seu centróide, ele é "estranho" em relação ao grupo.

### 🧠 Como detectar anomalias com DBSCAN?
O DBSCAN (agrupamento por densidade) já faz isso naturalmente:

Ele rotula automaticamente como outliers os pontos que não têm vizinhos suficientes dentro de uma certa distância (eps).

Esses pontos recebem o rótulo -1 (ruído).

➡️ Intuição: se um ponto está isolado, não faz parte de nenhuma região densa, então é uma anomalia.

In [ ]:
### Seção 6: Detecção de anomalias com K-Means

from sklearn.metrics import pairwise_distances_argmin_min

# Calcular distâncias para centros mais próximos
closest, distances = pairwise_distances_argmin_min(X, kmeans.cluster_centers_)

# Definir limite como os 5% mais distantes
threshold = np.percentile(distances, 95)
anomalies = distances > threshold

# Visualizar anomalias
plt.scatter(X[:, 0], X[:, 1], c='lightgray')
plt.scatter(X[anomalies, 0], X[anomalies, 1], color='red')
plt.title('Anomalias detectadas (top 5% mais distantes dos centros)')
plt.show()

### Exercício:
# - Mude o threshold (ex: 90%, 99%).
# - Compare a quantidade de anomalias.
# - Em quais situações isso seria útil no mundo real?

In [ ]:
### Desafio final:
# - Escolha um dataset real do UCI (ex: Wine, Breast Cancer, Mall Customers)
# - Aplique K-Means, DBSCAN e Hierárquico
# - Compare os agrupamentos e visualize com PCA
# - Proponha uma estratégia de detecção de anomalias